- Based on https://pygad.readthedocs.io/en/latest/

In [81]:
import pygad
import numpy as np

y = f(w1:w6) = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + w6x6
where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7) and y=44

In [82]:
function_inputs = [4,-2,3.5,5,-11,-4.7]
desired_output = 44

In [83]:
def fitness_func(ga_instance, solution, solution_idx):
    output = np.sum(solution*function_inputs)
    fitness = 1.0 / np.abs(output - desired_output)
    return fitness

In [84]:
def blend_crossover_pygad(parents, offspring_size, ga_instance):
    """
    A blend crossover function compatible with PyGAD.
    
    Parameters:
    - parents: The selected parents for crossover.
    - offspring_size: The size of the offspring to be produced.
    - ga_instance: The instance from the pygad.GA class.
    """
    alpha=0.5
#     - alpha: The alpha parameter controlling the diversity of offspring (not passed directly, defined inside).
    # Initialize an empty offspring array
    offspring = np.empty(offspring_size)
    
    # The number of parents to be selected for crossover
    num_parents = parents.shape[0]
    
    for k in range(offspring_size[0]):
        # Randomly selecting two parents
        parent1_idx, parent2_idx = np.random.choice(num_parents, 2, replace=False)
        parent1, parent2 = parents[parent1_idx], parents[parent2_idx]
        
        # Calculate the range I between parent genes
        I = np.abs(parent2 - parent1)
        
        # Calculate lower and upper bounds for each gene
        lower_bound = np.minimum(parent1, parent2) - I * alpha
        upper_bound = np.maximum(parent1, parent2) + I * alpha
        
        # Generate offspring by choosing a random value within the bounds for each gene
        offspring[k, :] = np.random.uniform(lower_bound, upper_bound)
    
    return offspring

In [85]:
a = 0.8
b = 0.9

def self_mutation(offspring, ga_instance):
    """
    Apply self-mutation to offspring with mutation parameters as part of the solution vector.
    
    Parameters:
    - offspring: The offspring to be mutated (2D NumPy array where each row is an individual).
    - ga_instance: The instance from the pygad.GA class.
    """
    # Assuming 'a' and 'b' are stored in the GA instance for easy access

    
    # Iterate through each individual in the offspring
    for individual in offspring:
        # Extract mutation probabilities from the individual
        mutation_probability = individual[-2]
        component_mutation_probability = individual[-1]
        
        # Decide if this individual will mutate
        if np.random.rand() < mutation_probability:
            # Apply component-wise mutation
            for i in range(len(individual) - 2):  # Exclude mutation probabilities
                if np.random.rand() < component_mutation_probability:
                    # Mutate this component
                    individual[i] = np.random.uniform(a, b)
                    
            # Optionally, mutate the mutation probabilities themselves
            # Ensure they remain within reasonable bounds [0.01, 0.99]
#             if np.random.rand() < component_mutation_probability:
#                 individual[-2] = np.clip(np.random.uniform(0, 1), 0.01, 0.99)
#             if np.random.rand() < component_mutation_probability:
#                 individual[-1] = np.clip(np.random.uniform(0, 1), 0.01, 0.99)
                
    return offspring

In [86]:
last_gen_population = None
last_gen_fitness = None

def merge_top_individuals(ga_instance):
    global last_gen_population, last_gen_fitness

    # Get the current population and their fitness scores
    current_population = ga_instance.population
    current_fitness = ga_instance.last_generation_fitness

    # Calculate the number of individuals to keep from each generation (40%)
    num_to_keep = int(0.4 * len(current_fitness))

    if last_gen_population is not None and last_gen_fitness is not None:
        # Sort the last generation by fitness and select the top 40%
        sorted_indices_last = np.argsort(last_gen_fitness)[-num_to_keep:]
        top_last_gen = last_gen_population[sorted_indices_last]

        # Sort the current generation by fitness and select the top 40%
        sorted_indices_current = np.argsort(current_fitness)[-num_to_keep:]
        top_current_gen = current_population[sorted_indices_current]

        # Merge the top individuals from both generations
        merged_population = np.vstack((top_last_gen, top_current_gen))

        # If the merged population is smaller than the original population size, fill the gap
        if len(merged_population) < len(current_population):
            additional_indices = np.random.choice(range(len(current_population)), size=(len(current_population) - len(merged_population)), replace=False)
            additional_individuals = current_population[additional_indices]
            merged_population = np.vstack((merged_population, additional_individuals))

        # Update the GA instance's population with the merged population
        ga_instance.population = merged_population

    # Update the last_gen_population and last_gen_fitness for the next call
    last_gen_population = current_population.copy()
    last_gen_fitness = current_fitness.copy()

def on_generation(ga_instance):
    merge_top_individuals(ga_instance)

In [87]:
fitness_function = fitness_func

num_generations = 50
num_parents_mating = 4

sol_per_pop = 8
num_genes = len(function_inputs)

init_range_low = -2
init_range_high = 5

keep_parents = int(0.4*num_parents_mating)

parent_selection_type = "sss"
crossover_type = "blend"

mutation_type = "random"
mutation_percent_genes = 10

In [89]:
ga_instance = pygad.GA(num_generations=num_generations,
                       on_generation = on_generation,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       keep_parents=keep_parents,
                       crossover_type=blend_crossover_pygad,
                       mutation_type=self_mutation,
                       parent_selection_type=parent_selection_type,
                       mutation_percent_genes=mutation_percent_genes)  # Example bounds)

In [90]:
ga_instance.run()

In [91]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

prediction = np.sum(np.array(function_inputs)*solution)
print("Predicted output based on the best solution : {prediction}".format(prediction=prediction))

Parameters of the best solution : [2.6164597  0.27931273 4.06085659 4.27257542 0.0116207  0.28835822]
Fitness value of the best solution = 113739.54158137238
Predicted output based on the best solution : 43.99997715261197
